In [1]:
!pip install transformers
!pip install torchmetrics
!pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 9.3 MB/s 
     |████████████████████████████████| 6.6 MB 64.5 MB/s 
     |████████████████████████████████| 596 kB 57.3 MB/s 
     |████████████████████████████████| 101 kB 11.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 419 kB 32.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 585 kB 34.1 MB/s 
     |████████████████████████████████| 140 kB 69.6 MB/s 
     |████████████████████████████████| 1.1 MB 63.3 MB/s 
     |████████████████████████████████| 271 kB 73.0 MB/s 
     |███████████████████████████

In [2]:
from google.colab import drive
from torch import nn
from transformers import BertTokenizer, BertModel
import torch
import csv
from torch.utils.data import Dataset
from transformers import BertTokenizer, BertModel, BertForMaskedLM, AdamW
from torchmetrics import Accuracy
import os
from torch.utils.data.dataloader import DataLoader
from torch.nn.utils.rnn import pad_sequence
import tqdm
import time
import math
import numpy as np
from transformers import DataCollatorForLanguageModeling
import pytorch_lightning as pl

In [3]:
from google.colab import drive
drive.mount('/content/drive')

data_path = '/content/drive/MyDrive/data'

train_data_path = f'{data_path}/train'
true_train_path = f'{train_data_path}/true.csv'
false_train_path = f'{train_data_path}/false.csv'

test_data_path = f'{data_path}/test'
true_test_path = f'{test_data_path}/true.csv'
false_test_path = f'{test_data_path}/false.csv'

Mounted at /content/drive


In [4]:


class TextClassificationModel(nn.Module):

    def __init__(self, embed_dim=768, num_class=1):
        super(TextClassificationModel, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased").cuda()
        self.fc = nn.Linear(embed_dim, num_class).cuda()
        self.sigmoid = nn.Sigmoid().cuda()

    def forward(self, input_ids, attention_masks):
        inputs = {
            "input_ids": input_ids.cuda(), 
            "attention_mask" : attention_masks.cuda()
        }
        outputs = self.bert(**inputs)
        output = outputs.last_hidden_state
        output = output[:,0,:]
        output = self.fc(output)
        return self.sigmoid(output)

# class BertLM(pl.LightningModule):

#     def __init__(self, class_name):
#         super().__init__()
#         self.bert = BertForMaskedLM.from_pretrained('bert-base-uncased')
#         self.epoch_number = 0
#         self.class_name = class_name

#     def forward(self, input_ids, labels):
#         return self.bert(input_ids=input_ids,labels=labels)

#     def training_step(self, batch, batch_idx):
#         input_ids = batch["input_ids"]
#         labels = batch["labels"]
#         outputs = self(input_ids=input_ids, labels=labels)
#         loss = outputs[0]
#         return {"loss": loss}

#     def training_epoch_end(self, outputs):
#         super().training_epoch_end(outputs)
#         mean_loss = 0
#         n_batch  = len(outputs)
#         for i in range(n_batch):
#             mean_loss += outputs[i]['loss'].cpu().numpy() / n_batch
#         print(f"End of epoch {self.epoch_number} with mean loss '{mean_loss}' on label {self.class_name}.", "fine_tuning")
#         self.epoch_number += 1
#     def configure_optimizers(self):
#         return AdamW(self.parameters(), lr=1e-5)

# class BertLMPred(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.bert = BertForMaskedLM.from_pretrained('bert-base-uncased')

#     def forward(self, input_ids, labels=None):
#         return self.bert(input_ids=input_ids,labels=labels)


In [5]:
class SpellCheckingDataset(Dataset):

    def __init__(self, tokenizer, data_paths, labels, batch_size=32):
        self.dataset = []
        for i in range(len(data_paths)):
          data_path = data_paths[i]
          with open(data_path, 'r', encoding='utf-8') as file:
              data = csv.reader(file)
              for item in data:
                self.dataset.append((item[0], labels[i]))
        self.tokenizer = tokenizer
        self.batch_size = batch_size

    def __getitem__(self, idx):
        if (self.tokenizer == None):
          raise Exception('Tokenizer cannot be null')

        tweet, label = self.dataset[idx]
        tokenized_tweet = self.tokenizer(tweet)
        input_ids = tokenized_tweet['input_ids']
        attention_mask = tokenized_tweet['attention_mask']

        return input_ids, attention_mask, label

    def __len__(self):
      return len(self.dataset)

class LMSpellCheckingDataset(Dataset):

    def __init__(self, tokenizer, data_paths, labels, batch_size=32):
        self.dataset = []
        for i in range(len(data_paths)):
          data_path = data_paths[i]
          with open(data_path, 'r', encoding='utf-8') as file:
              data = csv.reader(file)
              for item in data:
                self.dataset.append((item[0], labels[i]))
        self.tokenizer = tokenizer
        self.batch_size = batch_size

    def __getitem__(self, idx):
        if (self.tokenizer == None):
          raise Exception('Tokenizer cannot be null')

        tweet, label = self.dataset[idx]
        tokenized_tweet = self.tokenizer(tweet)
        input_ids = tokenized_tweet['input_ids']
        attention_mask = tokenized_tweet['attention_mask']

        return torch.tensor(input_ids)

    def __len__(self):
      return len(self.dataset)

In [6]:
# CHECKPOINTS_DIR = '/content/models'

def pad_batched_sequence(batch):
    
    input_ids = [torch.tensor(item[0]) for item in batch]
    # print('input ids')
    # print(input_ids)

    attention_masks =  [torch.tensor(item[1]) for item in batch]
    # print('attention masks')
    # print(attention_masks)

    input_ids = pad_sequence(input_ids, padding_value=0, batch_first=True)
    # print('pad input ids')
    # print(input_ids)

    attention_masks = pad_sequence(attention_masks, 
                                   padding_value=0, 
                                   batch_first=True)
    # print('pad attention masks')
    # print(attention_masks)

    labels = None
    
    if batch[0][2] is not None:
        labels = torch.tensor([[item[2]] for item in batch]).double().cuda()
    
    return input_ids.cuda(), attention_masks.cuda(), labels

class SpellCheckingTrainer():

  def __init__(self,
               model,
               train_dataset,
               save_data_path,
               batch_size=32,
               epochs=20,
               lr=0.001):
        
    self.model = model
    self.epochs = epochs
    self.batch_size = batch_size
    self.train_loader = DataLoader(train_dataset, 
                                   batch_size=batch_size, 
                                   shuffle=True, 
                                   drop_last=True, 
                                   collate_fn=pad_batched_sequence)
    self.save_path = save_data_path
    self.loss_function = nn.BCELoss()
    self.optimizer = torch.optim.Adam(list(self.model.parameters()), lr=lr)        
    self.accuracy = Accuracy(num_classes=1)

  def train_one_epoch(self, epoch_index):
    running_loss = 0.
    running_accuracy = 0.
    last_loss = 0.
    threshold = torch.tensor([0.5]).cuda()

    for i, data in tqdm.tqdm(enumerate(self.train_loader), 
                             total=len(self.train_loader)):
        # Every data instance is an input + label pair
        input_ids, attention_masks, labels = data

        # Zero your gradients for every batch!
        self.optimizer.zero_grad()

        # Make predictions for this batch
        outputs = self.model(input_ids, attention_masks)

        # Compute the loss and its gradients
        loss = self.loss_function(outputs.double(), labels.double())
        loss.backward()

        # Adjust learning weights
        self.optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        
        result = (outputs > threshold).float() * 1
        running_accuracy += torch.sum(result == labels) / self.batch_size
        
        if i % 10 == 9:
            last_loss = running_loss / 10 # loss per batch
            last_accuracy = running_accuracy / 10 # accuracy per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            print('  batch {} accuracy: {}'.format(i + 1, last_accuracy))
            running_loss = 0.
            running_accuracy = 0.

    return last_loss


  def train(self):
    epoch_number = 0
    for epoch in range(self.epochs):
        print('EPOCH {}:'.format(epoch_number + 1))

        # Make sure gradient tracking is on, and do a pass over the data
        self.model.train(True)
        avg_loss = self.train_one_epoch(epoch_number)

        # We don't need gradients on to do reporting
        self.model.train(False)

        epoch_number += 1
    
    torch.save(self.model.state_dict(), self.save_path)
    
          
          

In [7]:


tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

true_label = 1
false_label = 0

true_train_dataset = SpellCheckingDataset(tokenizer, [true_train_path], [true_label])
false_train_dataset = SpellCheckingDataset(tokenizer, [false_train_path], [false_label])

test_dataset = SpellCheckingDataset(tokenizer, 
                                    [true_test_path, false_test_path],
                                    [true_label, false_label])

true_saved_model_path = '/content/drive/MyDrive/models/true_bert.berm_lm'
false_saved_model_path = '/content/drive/MyDrive/models/false_bert.berm_lm'

true_trainer = SpellCheckingTrainer(TextClassificationModel(), 
                                    true_train_dataset,
                                    true_saved_model_path)

false_trainer = SpellCheckingTrainer(TextClassificationModel(), 
                                     false_train_dataset,
                                     false_saved_model_path)
 

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.trans

In [ ]:
true_trainer.train()
false_trainer.train()




EPOCH 1:


  2%|▏         | 10/403 [00:03<02:13,  2.95it/s]

  batch 10 loss: 0.0809702505817
  batch 10 accuracy: 0.90625


  5%|▍         | 20/403 [00:06<02:03,  3.09it/s]

  batch 20 loss: 9.74941587150786e-05
  batch 20 accuracy: 1.0


...

 52%|█████▏    | 210/403 [01:14<01:06,  2.92it/s]

  batch 210 loss: 3.3333954101563583e-06
  batch 210 accuracy: 1.0


 55%|█████▍    | 220/403 [01:18<01:03,  2.90it/s]

  batch 220 loss: 3.255909160827826e-06
  batch 220 accuracy: 1.0


 56%|█████▌    | 224/403 [01:19<01:03,  2.81it/s]